In [1]:
class Atmospheric():
    
    def round_date(date,xhour):
        """
        rounds a date of to the closest 'x' hours
        """
        y = date.get('year')
        m = date.get('month')
        d = date.get('day')
        H = date.get('hour')
        HH = H.divide(xhour).round().multiply(xhour)
        return date.fromYMD(y,m,d).advance(HH,'hour')
  
    def round_month(date):
        """
        round date to closest month
        """
        # start of THIS month
        m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

        # start of NEXT month
        m2 = m1.advance(1,'month')

        # difference from date
        d1 = ee.Number(date.difference(m1,'day')).abs()
        d2 = ee.Number(date.difference(m2,'day')).abs()

        # return closest start of month
        return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



    def water(geom,date):
        """
        Water vapour column above target at time of image aquisition.

        (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull. 
        Amer. Meteor. Soc., 77, 437-471)
        """

        # Point geometry required
        centroid = geom.centroid()

        # H2O datetime is in 6 hour intervals
        H2O_date = Atmospheric.round_date(date,6)

        # filtered water collection
        water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

        # water image
        water_img = ee.Image(water_ic.first())

        # water_vapour at target
        water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

        # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
        water_Py6S_units = ee.Number(water).divide(10)                                   

        return water_Py6S_units


    def ozone(geom,date):
        """
        returns ozone measurement from merged TOMS/OMI dataset

        OR

        uses our fill value (which is mean value for that latlon and day-of-year)

        """

        # Point geometry required
        centroid = geom.centroid()

        def ozone_measurement(centroid,O3_date):
            # filtered ozone collection
            ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

            # ozone image
            ozone_img = ee.Image(ozone_ic.first())

            # ozone value IF TOMS/OMI image exists ELSE use fill value
            ozone = ee.Algorithms.If(ozone_img,\
            ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
            ozone_fill(centroid,O3_date))

            return ozone

        def ozone_fill(centroid,O3_date):
            """
            Gets our ozone fill value (i.e. mean value for that doy and latlon)

            you can see it
            1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
            2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

            """

            # ozone fills (i.e. one band per doy)
            ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

            # day of year index
            jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
            doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

            # day of year image
            fill_image = ee.Image(ozone_fills.get(doy_index))

            # return scalar fill value
            return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

        # O3 datetime in 24 hour intervals
        O3_date = Atmospheric.round_date(date,24)

        # TOMS temporal gap
        TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')  

        # avoid TOMS gap entirely
        ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

        # fix other data gaps (e.g. spatial, missing images, etc..)
        ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

        #convert to Py6S units 
        ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )                             

        return ozone_Py6S_units


    def aerosol(geom,date):
        """
        Aerosol Optical Thickness.

        try:
          MODIS Aerosol Product (monthly)
        except:
          fill value
        """

        def aerosol_fill(date):
            """
            MODIS AOT fill value for this month (i.e. no data gaps)
            """
            return ee.Image('users/samsammurphy/public/AOT_stack')\
                   .select([ee.String('AOT_').cat(date.format('M'))])\
                   .rename(['AOT_550'])


        def aerosol_this_month(date):
            """
            MODIS AOT original data product for this month (i.e. some data gaps)
            """
            # image for this month
            img =  ee.Image(\
                          ee.ImageCollection('MODIS/006/MOD08_M3')\
                            .filterDate(Atmospheric.round_month(date))\
                            .first()\
                         )

            # fill missing month (?)
            img = ee.Algorithms.If(img,\
                                   # all good
                                   img\
                                   .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                                   .divide(1000)\
                                   .rename(['AOT_550']),\
                                  # missing month
                                    aerosol_fill(date))

            return img    


        def get_AOT(AOT_band,geom):
            """
            AOT scalar value for target
            """  
            return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                     geometry=geom.centroid())\
                                    .get('AOT_550')


        after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

        AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

        AOT = get_AOT(AOT_band,geom)

        AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
        # i.e. check reduce region worked (else force fill value)

        return AOT

In [2]:
import geemap
import ee
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd

import numpy as np
from glob import glob
import urllib.request
from matplotlib.pyplot import imshow
from PIL import Image,ImageDraw,ImageFont

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
from Py6S import *

In [3]:
def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('RWS')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B11']).rename('MNDWI')
    mgrn = img.select(['B3','B4','B8']).divide(10000).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        '(MNDWI > 0.3) && (MGRN < 0.15)',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)

def apply_rws(img):
    mask = img.select('RWS').eq(1)
    masked_img = img.updateMask(mask)
    return masked_img

def get_labels(img):
    cluster =  img.select('cluster')
    cluster_vector = cluster.reduceToVectors(reducer = ee.Reducer.countEvery(),geometry=img.geometry(),bestEffort=True,scale=90)
    labels = ee.List(cluster_vector.aggregate_array('label')).distinct().sort()
    return labels

def add_kmeans(img):
    rws_mask = img.select('RWS').eq(1)
    rgb_img = img.select(['B4', 'B3', 'B2']).updateMask(rws_mask)
    
    train_samples = rgb_img.sample(scale=10,numPixels=2000,seed=42,tileScale=4,dropNulls=True)
    k_means = ee.Clusterer.wekaKMeans(8).train(train_samples)
    img_cluster = rgb_img.cluster(k_means).rename('cluster')
    return img.addBands(img_cluster)

def plot_mask(data,layer_name):
    img = data.select(layer_name)
    img_mask = img.updateMask(img.eq(1))
    Map.addLayer(img_mask, {'palette': 'red'}, f'{layer_name}')
    
def plot_rgb(img,text,bands = ['B4', 'B3', 'B2']):
    viz_rgb = {'bands': bands,'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' RGB')
    
def plot_false(img,text):
    viz_rgb = {'bands': ['B6', 'B5', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' False')
    
def get_dates(data):
    date= ee.Image(data).get('GENERATION_TIME')
    return date

def add_mnws(img):

    band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
    multiband_img = img.select(band_names)
    cluster_img = img.select('cluster')

    samples = multiband_img.addBands(cluster_img).stratifiedSample(numPoints=100,classBand='cluster',scale=10,tileScale=4,seed=42,dropNulls=True)

    nws_list = []
    for label in list(range(0,8)):

        scores = []

        for band in band_names:
            img_band_mu = samples.filter(ee.Filter.eq('cluster',label)).aggregate_mean(band)
            img_band_std = samples.filter(ee.Filter.eq('cluster',label)).aggregate_total_sd(band)

            img_band_raw = multiband_img.select(band)
            img_band_score = img_band_raw.subtract(img_band_mu).divide(img_band_std).pow(2).rename(f'{label} {band}')
            
            scores.append(img_band_score)

        nws = ee.Image.cat(scores).reduce('sum').divide(len(band_names)).sqrt().rename('NWS')
        nws_list.append(nws)

    mnws = ee.Image.cat(nws_list).reduce('min').rename('MNWS')
    return mnws

def get_wd(img):
    img_mnws = img.select('MNWS')
    wd_i = img.expression('MNWS < 3',{'MNWS':img_mnws})
    return wd_i

def count_water_pixel(img):
    count = img.select('RWS').gt(0).reduceRegion(ee.Reducer.sum(),scale=10,maxPixels=1e5,bestEffort=True,tileScale=4).values().get(0)
    return img.set({'water_pixel': count})

def export_frames(img_col,dir_name,annot_list):
    
    print(f'Exporting {len(annot_list)} frames...')
    args = {'dimensions': 480,'bands': ['B11', 'B8', 'B3'],'gain': [0.08, 0.08, 0.08],'format':'png'}
    url = img_col.getFilmstripThumbURL(args)
    
    os.makedirs(dir_name,exist_ok=True)

    saved_strips = urllib.request.urlretrieve(url,f'{dir_name}.png')[0]
    
    img_arr = np.array(Image.open(saved_strips))
    img_stack = img_arr.reshape(len(annot_list),img_arr.shape[0]//len(annot_list),img_arr.shape[1],img_arr.shape[2])
    
    for i in range(len(annot_list)):
        img_pil = Image.fromarray(img_stack[i])
        ImageDraw.Draw(img_pil).text((0, 0),annot_list[i],font = ImageFont.truetype("arial.ttf", 20),fill=(0,0,0))
        fn = f"{dir_name}/{dir_name.split('/')[-1]}_{i}.png"  
        img_pil.save(fn)
        
    os.remove(saved_strips)
    print(f'{len(annot_list)} frames exported in {dir_name}')

In [ ]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
# Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/ramsar_stlucia_bbox.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")

Map


In [ ]:
# jrc_gsw_s = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq('year',2017)).first().clip(poly_geom)
# Map.addLayer(jrc_gsw_s,{'min': 0, 'max': 3,'palette':['cccccc','ffffff','99d9ea','0000ff']},'JRC GSWS-S 2017')


# srtm_30m = ee.Image("USGS/SRTMGL1_003").clip(poly_geom)
srtm_30m_slope = ee.Terrain.slope(srtm_30m)
# Map.addLayer(srtm_30m,{'min': 0, 'max': 10},'srtm')
Map.addLayer(srtm_30m_slope.lte(5),{'min': 0, 'max': 1},'slope')

# jrc_gsw_s_m = ee.ImageCollection("JRC/GSW1_1/MonthlyHistory").filter(ee.Filter.eq('year',2017)).filter(ee.Filter.eq('month',1)).first().clip(poly_geom)
# Map.addLayer(jrc_gsw_s_m,{'min': 0, 'max': 2,'palette':['cccccc','ffffff','0000ff']},'JRC GSWS-S 2017 January')

In [ ]:
#get Sentinel-2 Level-1C collection for 2017

cl_pct = 60

s2_col = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(poly_geom).filterDate('2017-01-01','2017-12-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cl_pct))

# #filter duplicated products 
s2_col_list = s2_col.toList(s2_col.size())

print('Nr. images found: ', s2_col.size().getInfo())


In [ ]:
unique_dates = s2_col_list.map(get_dates).distinct()

#iterave over collection and create mosaic based on similar datetime
col = []
for i in range(unique_dates.size().getInfo()):
    unique_date = unique_dates.get(i)
    date_day = ee.Date(unique_dates.get(i)).format('yyyy-MM-dd')
    date_month = ee.Date(unique_dates.get(i)).format('yyyy-MM')
    similar_tiles = s2_col.filter(ee.Filter.eq('GENERATION_TIME', unique_date)).mosaic().clip(poly_geom).set({'DateDay':date_day,'Month':date_month})
    col.append(similar_tiles)
    
s2_mosaic_col_a = ee.ImageCollection(col)

#iterate over mosaic again and merge based on similar days
date_days = ee.List(s2_mosaic_col_a.aggregate_array('DateDay')).getInfo()
days,count = np.unique(date_days, return_counts=True)
dups = days[count>1]
non_dups = np.setdiff1d(days,dups)

dup_mos = ee.ImageCollection([s2_mosaic_col_a.filter(ee.Filter.eq('DateDay',dup)).mosaic().clip(poly_geom).set({'DateDay':dup}) for dup in dups.tolist()])
non_dup_mos = s2_mosaic_col_a.filter(ee.Filter.inList('DateDay', non_dups.tolist()))

s2_mosaic_col_clean = non_dup_mos.merge(dup_mos).sort('DateDay')

print('Nr. mosaics created: ', len(date_days),len(days))

In [ ]:
# #export mosaics as png
# dir_name = './data/ts_animation/2019_cl60'
# export_frames(s2_mosaic_col_clean,dir_name,days.tolist())

In [ ]:
#select best mosaics 
cleaned_dir_name = './data/ts_animation/2017_cl60_cleaned_v2'
selected_indices = np.sort(np.array(list(map(lambda x:x.split('_')[-1].split('.')[0],os.listdir(cleaned_dir_name)))).astype(int))
selected_days = days[selected_indices].tolist()

#filter best mosaic from collection
s2_mosaic_best = s2_mosaic_col_clean.filter(ee.Filter.inList('DateDay', selected_days))
s2_mosaic_best_list = s2_mosaic_best.toList(s2_mosaic_best.size())
print(s2_mosaic_best.size().getInfo(),'mosaics selected')

In [ ]:
import math
import datetime
band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B10','B11','B12']
toa = s2_mosaic_best.select(band_names).first().divide(10000)
info = s2_col.first().getInfo()['properties']
scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

date = ee.Date(str(scene_date.date()))
geom = ee.Geometry.Point(-27.93186,32.48897)

In [ ]:
h2o = Atmospheric.water(geom,date).getInfo()
o3 = Atmospheric.ozone(geom,date).getInfo()
aot = Atmospheric.aerosol(geom,date).getInfo()

In [ ]:
SRTM = ee.Image("USGS/SRTMGL1_003")# Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = poly_geom).get('elevation').getInfo()
km = alt/1000 # i.e. Py6S uses units of kilometers

In [ ]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

In [ ]:
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """

    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    
    return Wavelength(bandSelect[bandname])

def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
   
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    
    return rad

def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    
    return ref

In [ ]:
# surface reflectance rgb
b = surface_reflectance('B2')
g = surface_reflectance('B3')
r = surface_reflectance('B4')
ref = r.addBands(g).addBands(b)

In [ ]:
raw_dates = s2_col_list.map(get_dates).map(lambda x:ee.Date(x).format('yyyy-MM-dd')).getInfo()
sun_ele = s2_col.aggregate_array('MEAN_SOLAR_ZENITH_ANGLE').getInfo()
sun_azi = s2_col.aggregate_array('MEAN_SOLAR_AZIMUTH_ANGLE').getInfo()
data = np.array( [raw_dates,sun_ele,sun_azi] )

sun_data_clean = {}
for d in selected_days:
    index = np.where(data[0]==d)[0]
    sun_ele_mu = data[1][index].astype(float).mean()
    sun_azi_mu = data[2][index].astype(float).mean()
    sun_info = [sun_ele_mu,sun_azi_mu]
    sun_data_clean[d]=sun_info
    
sun_data_df = pd.DataFrame(sun_data_clean).T
sun_data_df.columns = ['mean_sun_zen_angle','mean_sun_azi_angle']
sun_data_df.to_csv('./data/sun_data.csv')

In [ ]:
plot_rgb(toa.multiply(10000),'toa')

In [ ]:
plot_rgb(ref.multiply(10000),'sr')

In [ ]:
#create median composite collection
band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B10','B11','B12']

months = [f'2017-{str(i).zfill(2)}' for i in range(1,13)]
new_group = []
for m in months:
    group = []
    for d in selected_days:
        if m in d:
            group.append(d)
    new_group.append(group)
    
monthly_col = []
for g in new_group:
    if len(g)>1:
        s2_mosaic_month = s2_mosaic_best.filter(ee.Filter.inList('DateDay', g)).select(band_names).median()
    else:
        s2_mosaic_month = s2_mosaic_best.filter(ee.Filter.inList('DateDay', g)).select(band_names).first()
    monthly_col.append(s2_mosaic_month)
monthly_col = ee.ImageCollection(monthly_col)
monthly_col_list = monthly_col.toList(monthly_col.size())

In [ ]:
mnws_rws_25feb = add_mnws(add_kmeans(add_rws(s2_mosaic_best.first())))
mnws_wiw_25feb = add_mnws(add_kmeans(add_wiw(s2_mosaic_best.first())))
file_path = './data/boundaries/landcover_25feb.geojson'
with open(file_path) as f:
    coords = json.load(f)
    features = geojson_to_ee(coords)
    labels = features.aggregate_array('type').distinct()
    labels_int = ee.List([1,2,3,4,5,6])
    features_remapped = features.remap(labels,labels_int,'type')
    label_class = dict(zip(labels_int.getInfo(),labels.getInfo()))

In [ ]:
mnws_rws_25feb_samples = mnws_rws_25feb.sampleRegions(collection= features_remapped,properties= ['type'],scale= 10,tileScale=4)
mnws_wiw_25feb_samples = mnws_wiw_25feb.sampleRegions(collection= features_remapped,properties= ['type'],scale= 10,tileScale=4)

In [ ]:
mnws_values = mnws_rws_25feb_samples.aggregate_array('MNWS').getInfo()
labels_values = mnws_rws_25feb_samples.aggregate_array('type').getInfo()

data = pd.DataFrame(np.array([mnws_values,labels_values])).T
data.columns = ['MNWS','label']
data['label'] = data['label'].astype(int)
data['label'] = data['label'].map(label_class)
data.boxplot(column='MNWS',by='label',grid=False,rot=45, fontsize=20,figsize=(10,5))

for lab in data.label.unique():
    print(data[data['label']==lab].describe().loc[['min','mean','max','std']].T,lab)

In [ ]:
mnws_values = mnws_wiw_25feb_samples.aggregate_array('MNWS').getInfo()
labels_values = mnws_wiw_25feb_samples.aggregate_array('type').getInfo()

data = pd.DataFrame(np.array([mnws_values,labels_values])).T
data.columns = ['MNWS','label']
data['label'] = data['label'].astype(int)
data['label'] = data['label'].map(label_class)
data.boxplot(column='MNWS',by='label',grid=False,rot=45, fontsize=20,figsize=(10,5))

for lab in data.label.unique():
    print(data[data['label']==lab].describe().loc[['min','mean','max','std']].T,lab)

In [ ]:
#map MNWS timeseries with RWS (original)
mnws_ts_rws = s2_mosaic_best.map(add_rws).map(add_kmeans).map(add_mnws)

#map MNWS timeseries with WIW 
mnws_ts_wiw = s2_mosaic_best.map(add_wiw).map(add_kmeans).map(add_mnws)

In [ ]:
#calculate IWF with RWS
img_wf_rws = mnws_ts_rws.map(get_wd).reduce(reducer=ee.Reducer.sum(),parallelScale=8).divide(len(selected_days))

#calculate IWF with WIW
img_wf_wiw = mnws_ts_wiw.map(get_wd).reduce(reducer=ee.Reducer.sum(),parallelScale=8).divide(len(selected_days))

In [ ]:
# export all bands stack to one geotiff
geemap.ee_export_image(img_wf_rws, filename='./data/iwf/iwf_rws_39.tif', scale=90, region=poly_geom, file_per_band=False)
geemap.ee_export_image(img_wf_wiw, filename='./data/iwf/iwf_wiw_39.tif', scale=90, region=poly_geom, file_per_band=False)

remove_zips = list(map(lambda x:os.remove(x),glob('./data/iwf/*.zip')))

In [ ]:
#export image collection as geotiff 
# geemap.ee_export_image_collection(mnws_ts, out_dir='./data/mnws_ts_wiw',region=poly_geom,scale=90)
# remove_zips = list(map(lambda x:os.remove(x),glob('./data/mnws_ts_wiw/*.zip')))

In [ ]:
# export image to drive
band_names = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']
aoi = ee_to_geojson(poly_geom)['coordinates'][0]

for i in range(len(selected_days[21:])):

    file_name = 'mosaic_'+selected_days[21:][i].replace('-','')
    data = ee.Image(s2_mosaic_best_list.get(i)).select(band_names)
    task = ee.batch.Export.image.toDrive(data, 
                                         description='mosaic',
                                         folder='GEE_output',
                                        fileNamePrefix=file_name,
                                        crs='EPSG:32736',
                                        scale=10,
                                        maxPixels=1e8,
                                        fileFormat='GeoTIFF',
                                        skipEmptyTiles=True,
                                        region=aoi)
    task.start()

In [ ]:
task.status()

In [ ]:
jrc_gsw_s = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq('year',2017)).first().clip(poly_geom)
srtm_30m = ee.Image("USGS/SRTMGL1_003").clip(poly_geom)
srtm_30m_slope = ee.Terrain.slope(srtm_30m)
jrc_gsw_s_m = ee.ImageCollection("JRC/GSW1_1/MonthlyHistory").filter(ee.Filter.eq('year',2017)).map(lambda x:x.clip(poly_geom)).toList(12)
jrc_gsw_s_m_list = [ee.Image(jrc_gsw_s_m.get(i)) for i in range(12)]

imgs2exp = [jrc_gsw_s,srtm_30m,srtm_30m_slope]+jrc_gsw_s_m_list
fnames = ['YearlyHistory_2017','srtm_30m','srtm_30m_slope'] + [f'MonthlyHistory_{i}' for i in list(range(1,13))]

aoi = ee_to_geojson(poly_geom)['coordinates'][0]

for i in range(len(fnames)):
    task = ee.batch.Export.image.toDrive(imgs2exp[i], 
                                         description='ancillary_data',
                                         folder='GEE_output',
                                         fileNamePrefix=fnames[i],
                                         crs='EPSG:32736',
                                         scale=10,
                                         maxPixels=1e8,
                                         fileFormat='GeoTIFF',
                                         skipEmptyTiles=True,
                                         region=aoi)
    task.start()

In [ ]:
# export image to drive

band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B10','B11','B12']

#export median composite
aoi = ee_to_geojson(poly_geom)['coordinates'][0]
task = ee.batch.Export.image.toDrive(s2_mosaic_best.select(band_names).median(), 
                                         description='mosaic_composite',
                                         folder='GEE_output',
                                         fileNamePrefix='4mosaics_median_composite_june',
                                         crs='EPSG:32736',
                                         scale=10,
                                         maxPixels=1e8,
                                         fileFormat='GeoTIFF',
                                         skipEmptyTiles=True,
                                         region=aoi)
task.start()

In [ ]:
task.status()